## Again Loading the Index and Creating a Chatbot with Gradio interface

### Retrieving API KEY

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
google_gemini_api_key = os.getenv("GOOGLE_GENAI_API_KEY")
print(google_gemini_api_key)

### Changing Embedded Model & LLM from OpenAI to Googl Gemini

In [ ]:
from llama_index.core import Settings
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.llms.google_genai import GoogleGenAI

Settings.embed_model = GoogleGenAIEmbedding(
    model_name="text-embedding-004",
    api_key=google_gemini_api_key,
    embed_batch_size=100
)

Settings.llm = GoogleGenAI(
    model="gemini-2.0-flash",
    api_key=google_gemini_api_key
)

### Creating Storage_Index & Loading Index

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

storage_index = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context=storage_index)

### Creating Chat_Engine

In [ ]:
chat_engine = index.as_chat_engine(verbose=True)

In [ ]:
chat_engine.chat("Hii")

### Chat Interface with Gradio

In [ ]:
# Wraps our chat history to format that gradio wants
def wrapper_chat_history(memory):
    chat_history = []
    for m in memory:
        if m.role in ['user', 'assistant'] and m.content is not None:
            chat_history.append(m.content)
    
    return chat_history

In [ ]:
wrapper_chat_history(chat_engine.chat_history)

In [ ]:
def converse(message, memory):
    response = chat_engine.chat(message=message)
    chat_history = wrapper_chat_history(chat_engine.chat_history)
    return response.response

In [ ]:
import gradio as gd

demo = gd.ChatInterface(fn=converse)
demo.launch(share=False)

In [ ]:
chat_engine.chat_history

In [ ]:
chat_engine.reset()